## Deliverable 2. Create a Customer Travel Destinations Map.

In [21]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [22]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sao Felix Do Xingu,BR,-6.6447,-51.9950,84.79,41,81,1.68,broken clouds
1,1,Souillac,MU,-20.5167,57.5167,73.78,78,40,9.22,scattered clouds
2,2,Albany,US,42.6001,-73.9662,58.57,90,100,1.01,overcast clouds
3,3,Pattoki,PK,31.0167,73.8500,98.47,8,4,1.23,clear sky
4,4,Kruisfontein,ZA,-34.0033,24.7314,54.88,90,100,9.89,moderate rain


In [60]:
city_data_df["Country"].replace('', np.nan, inplace=True)
city_data_df.dropna(subset=['Country'], inplace=True)

In [61]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 40
What is the maximum temperature you would like for your trip? 70


In [62]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Country"] != "")]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Albany,US,42.6001,-73.9662,58.57,90,100,1.01,overcast clouds
4,4,Kruisfontein,ZA,-34.0033,24.7314,54.88,90,100,9.89,moderate rain
5,5,Mataura,NZ,-46.1927,168.8643,41.49,94,72,7.54,light rain
8,8,Port Lincoln,AU,-34.7333,135.8667,57.90,62,1,15.23,clear sky
9,9,Hobart,AU,-42.8794,147.3294,45.27,83,6,5.75,clear sky


In [63]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                301
City                   301
Country                301
Lat                    301
Lng                    301
Max Temp               301
Humidity               301
Cloudiness             301
Wind Speed             301
Current Description    301
dtype: int64

In [65]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Albany,US,58.57,overcast clouds,42.6001,-73.9662,
4,Kruisfontein,ZA,54.88,moderate rain,-34.0033,24.7314,
5,Mataura,NZ,41.49,light rain,-46.1927,168.8643,
8,Port Lincoln,AU,57.90,clear sky,-34.7333,135.8667,
9,Hobart,AU,45.27,clear sky,-42.8794,147.3294,
10,Beidao,CN,60.75,overcast clouds,34.5686,105.8933,
11,Torbay,CA,52.88,broken clouds,47.6666,-52.7314,
12,Ketchikan,US,46.87,broken clouds,55.3422,-131.6461,
14,Saldanha,ZA,55.45,broken clouds,-33.0117,17.9442,
15,Hermanus,ZA,58.39,overcast clouds,-34.4187,19.2345,


In [66]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [71]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Kruisfontein,ZA,54.88,moderate rain,-34.0033,24.7314,Oyster Bay House Rental
5,Mataura,NZ,41.49,light rain,-46.1927,168.8643,Ellie's Villa
8,Port Lincoln,AU,57.90,clear sky,-34.7333,135.8667,Port Lincoln Hotel
9,Hobart,AU,45.27,clear sky,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
10,Beidao,CN,60.75,overcast clouds,34.5686,105.8933,Shell Tianshui Maiji District Erma Road Walkin...


In [72]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Kruisfontein,ZA,54.88,moderate rain,-34.0033,24.7314,Oyster Bay House Rental
5,Mataura,NZ,41.49,light rain,-46.1927,168.8643,Ellie's Villa
8,Port Lincoln,AU,57.90,clear sky,-34.7333,135.8667,Port Lincoln Hotel
9,Hobart,AU,45.27,clear sky,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
10,Beidao,CN,60.75,overcast clouds,34.5686,105.8933,Shell Tianshui Maiji District Erma Road Walkin...


In [74]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [76]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [77]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))